# 03 — Comparación de Estrategias de Prompts
Comparación de las 4 estrategias de prompting en el sistema RAG.

In [ ]:
import sys, json
from pathlib import Path
ROOT = Path("..").resolve()
sys.path.insert(0, str(ROOT))
from dotenv import load_dotenv
load_dotenv(ROOT / ".env")
print("Setup OK")

In [ ]:
from src.retrieval import search
QUESTION = "¿Cuáles son las principales estrategias de resistencia a la extorsión en América Latina?"
print(f"Question: {QUESTION}
")
chunks = search(QUESTION, top_k=5, strategy="small")
print(f"Retrieved {len(chunks)} chunks:")
for c in chunks:
    print(f"  [{c.score:.3f}] {c.title[:60]} ({c.year})")

In [ ]:
from src.generation import compare_strategies
results = compare_strategies(QUESTION, chunks)
for strat, res in results.items():
    print(f"
{'='*60}")
    print(f"STRATEGY: {res['strategy_label']}")
    print(f"Tokens: {res['total_tokens']} | Time: {res['elapsed_seconds']}s")
    print('-'*60)
    print(res['answer'][:800])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
strategies = list(results.keys())
token_counts = [results[s]['total_tokens'] for s in strategies]
answer_lens = [len(results[s]['answer']) for s in strategies]
x = np.arange(len(strategies))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.bar(x, token_counts, color=['#2196F3','#4CAF50','#FF9800','#9C27B0'])
ax1.set_xticks(x); ax1.set_xticklabels(strategies, rotation=15)
ax1.set_title('Total Tokens Used'); ax1.set_ylabel('Tokens')
ax2.bar(x, answer_lens, color=['#2196F3','#4CAF50','#FF9800','#9C27B0'])
ax2.set_xticks(x); ax2.set_xticklabels(strategies, rotation=15)
ax2.set_title('Answer Length (chars)'); ax2.set_ylabel('Characters')
plt.suptitle('Strategy Comparison Metrics')
plt.tight_layout()
plt.show()

In [ ]:
with open(ROOT / "eval" / "test_questions.json", encoding="utf-8") as f:
    eval_data = json.load(f)
questions_df_data = [{"id": q["id"], "category": q["category"], "difficulty": q["difficulty"], "question": q["question"][:80]} for q in eval_data["questions"]]
import pandas as pd
df_q = pd.DataFrame(questions_df_data)
print(f"Total test questions: {len(df_q)}")
print(f"
By category:
{df_q['category'].value_counts()}")
print(f"
By difficulty:
{df_q['difficulty'].value_counts()}")
df_q.head(10)